# COMP442 ASSIGNMENT 1

In [1]:
#imports
using Pkg; for p in ("Knet","IterTools"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Statistics: mean
using IterTools: ncycle
using WordTokenizers
using Iterators

ArgumentError: ArgumentError: Package Iterators not found in current path:
- Run `import Pkg; Pkg.add("Iterators")` to install the Iterators package.


In [2]:
cdictP = Dict()
cdictN = Dict()
wdict = Dict()
cdict = Dict()
w2i(x)= get!(wdict,x,1+length(wdict))
countN(x) =cdictN[x] = get(cdictN,x,0)+1
countP(x) =cdictP[x] = get(cdictP,x,0)+1

UNK = w2i("<unk>")
PAD = w2i("Burkipad")


2

In [3]:
#train Neg
train = []


train_neg= cd(readdir, "/home/burak/julia-1.0.1/Julia Work/aclImdb_v1/aclImdb/train/neg")
for f in train_neg  
    data = []
    open("/home/burak/julia-1.0.1/Julia Work/aclImdb_v1/aclImdb/train/neg/"*f,"r") do fi
        for line in eachline(fi)
            push!(data,line)
        end
#            wordids = lowercase.(split(filter(s->contains(),filter(s->!ispunct(s), data[1]))))

    #wordids = (lowercase.(tokenize(replace(data[1],"."=>""))))

        wordids = (replace(data[1],['#','@','\n','!','?',')','(','{','}','>','<','_']=>""))
        wordids = replace(wordids,['.']=>" .")
        wordids = replace(wordids,[',']=> " ,")
       wordids =  (lowercase.(split(wordids)))
    if(length(wordids)> 150)
            wordids = wordids[1:150]
        end
    while(length(wordids)<150)
        push!(wordids,"Burkipad")
        end
    wordids = w2i.(wordids)
    countN.(wordids)
    push!(train,(wordids,0))
    end

end

In [4]:
#train Pos



train_pos= cd(readdir, "/home/burak/julia-1.0.1/Julia Work/aclImdb_v1/aclImdb/train/pos")
for f in train_pos  
    data = []
    open("/home/burak/julia-1.0.1/Julia Work/aclImdb_v1/aclImdb/train/pos/"*f,"r") do fi
        for line in eachline(fi)
            
            push!(data,line)
        end
    #wordids = (lowercase.(tokenize(replace(data[1],"."=>""))))
        #wordids =(lowercase.(split(replace(data[1],['#','@','\n',',','.','!','?',')','(','{','}','>','<']=>""))))
    #wordids = (lowercase.(split(filter(s->!ispunct(s), data[1]))))
 wordids = (replace(data[1],['#','@','\n','!','?',')','(','{','}','>','<','_']=>""))
        wordids = replace(wordids,['.']=>" .")
        wordids = replace(wordids,[',']=> " ,")
       wordids =  (lowercase.(split(wordids)))
        if(length(wordids)> 150)
            wordids = wordids[1:150]
        end
        while(length(wordids)<150)
        push!(wordids,"Burkipad")
        end
    wordids = w2i.(wordids)
    countP.(wordids)
    push!(train,(wordids,0))   
    end

end


In [5]:
nposwords = sum(values(cdictP))
nnegwords = sum(values(cdictN))
nnegwords/length(cdictN)

28.018529587567244

In [6]:
length(train)

25000

In [7]:
#Frequency threshold 40 otherwise add values to UNK

In [7]:
th = 100

100

In [8]:
cdictP[1]=th
for i in collect(keys(cdictP))
    if(cdictP[i]<th)
        cdictP[1]=cdictP[1] +cdictP[i]
        delete!(cdictP,i)
    end
end

In [9]:
cdictN[1]=th
for i in collect(keys(cdictN))
    if(cdictN[i]<th)
        cdictN[1]=cdictN[1] +cdictN[i]
        delete!(cdictN,i)
    end
end

In [36]:
#Return the value of UNK if not found

In [10]:
getcdictP(x) = get(cdictP, x, cdictP[1]-th)     # unk if not found
getcdictN(x) = get(cdictN, x, cdictN[1]-th)     # unk if not found
w2i(x) = get(wdict, x, UNK)     # unk if not found


w2i (generic function with 1 method)

In [11]:
#probabilities
#P(c):
pPositive = 0.5
pNegative = 0.5
Ppos(x) =((getcdictP(x))/(nposwords))
Pneg(x) =((getcdictN(x))/(nnegwords))
genProcessPos(x) = pPositive*prod(log.(Ppos.(x)))
genProcessNeg(x) = pNegative*prod(log.(Pneg.(x)))

genProcessNeg (generic function with 1 method)

In [12]:
count = 0
for i=1:12500
    if (genProcessNeg(train[i][1])>genProcessPos(train[i][1]))
     count = count+1 
    end
end

for i=12501:25000
    if (genProcessNeg(train[i][1])<genProcessPos(train[i][1]))
        count = count+1 
    end
end
count

5070

In [13]:
count/25000

0.2028

In [16]:
#load dataset
test = []
#test Neg

test_neg= cd(readdir, "/home/burak/julia-1.0.1/Julia Work/aclImdb_v1/aclImdb/test/neg")
for f in test_neg  
    data = []
    open("/home/burak/julia-1.0.1/Julia Work/aclImdb_v1/aclImdb/test/neg/"*f,"r") do fi
        for line in eachline(fi)
            push!(data,line)
        end
  wordids = (replace(data[1],['#','@','\n','!','?',')','(','{','}','>','<','_']=>""))
        wordids = replace(wordids,['.']=>" .")
        wordids = replace(wordids,[',']=> " ,")
       wordids =  (lowercase.(split(wordids)))
        if(length(wordids)> 150)
            wordids = wordids[1:150]
        end
        while(length(wordids)<150)
        push!(wordids,"Burkipad")
        end
    wordids = w2i.(wordids)
    push!(test,(wordids,0))   
    end

end

#test Pos
test_pos= cd(readdir, "/home/burak/julia-1.0.1/Julia Work/aclImdb_v1/aclImdb/test/pos")
for f in test_pos  
    data = []
    open("/home/burak/julia-1.0.1/Julia Work/aclImdb_v1/aclImdb/test/pos/"*f,"r") do fi
        for line in eachline(fi)
            push!(data,line)
        end
        
        
        
wordids = (replace(data[1],['#','@','\n','!','?',')','(','{','}','>','<','_']=>""))
        wordids = replace(wordids,['.']=>" .")
        wordids = replace(wordids,[',']=> " ,")
       wordids =  (lowercase.(split(wordids)))
        if(length(wordids)> 150)
            wordids = wordids[1:150]
        end
        while(length(wordids)<150)
        push!(wordids,"Burkipad")
        end
    wordids = w2i.(wordids)
    push!(test,(wordids,1))   
    end

end

    

In [17]:
count = 0
for i=1:12500
    if (genProcessNeg(test[i][1])>genProcessPos(test[i][1]))
     count = count+1 
    end
end

for i=12501:25000
    if (genProcessNeg(test[i][1])<genProcessPos(test[i][1]))
        count = count+1 
    end
end
count

16073

In [18]:
count/25000

0.64292

In [26]:
getcdictP(x) = get(cdictP, x, 0)     # unk if not found# 50
getcdictN(x) = get(cdictN, x,0)     # unk if not found
w2i(x) = get(wdict, x, UNK)     # unk if not found
#numUnkP = cdictP[1]-40
#numUnkN = cdictN[1]-40

w2i (generic function with 1 method)

In [27]:
#probabilities
#P(c):
pPositive = 0.5
pNegative = 0.5
#P(wi|y)  number of occurance of wi with label y / number of all word with label y
# with smoothing number of occurance of wi with label y + alpha / number of all words with label y  + numWord * alpha
Ppos(x) =((getcdictP(x)+1)/(nposwords+length(cdictP)))
Pneg(x) =((getcdictN(x)+1)/(nnegwords+length(cdictN)))
genProcessPos(x) = pPositive*prod(log.(Ppos.(x)))
genProcessNeg(x) = pNegative*prod(log.(Pneg.(x)))

genProcessNeg (generic function with 1 method)

In [ ]:
#Neglecting the UNK

In [ ]:
getcdictP(x) = get(cdictP, x, MathConstants.e*((nposwords)-numUnkP))     # unk if not found # 65
getcdictN(x) = get(cdictN, x,MathConstants.e*((nnegwords)-numUnkN))     # unk if not found
w2i(x) = get(wdict, x, UNK)     # unk if not found
numUnkP = cdictP[1]-40
numUnkN = cdictN[1]-40

In [51]:
#probabilities
#P(c):
pPositive = 0.5
pNegative = 0.5
#P(wi|y)  number of occurance of wi with label y / number of all word with label y
# with smoothing number of occurance of wi with label y + alpha / number of all words with label y  + numWord * alpha
Ppos(x) =((getcdictP(x))/((nposwords)-numUnkP))
Pneg(x) =((getcdictN(x))/((nnegwords)-numUnkN))
genProcessPos(x) = pPositive*prod(log.(Ppos.(x)))
genProcessNeg(x) = pNegative*prod(log.(Pneg.(x)))

genProcessNeg (generic function with 1 method)